In [23]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from datetime import date

In [19]:
data = pd.read_csv('UpsampledData.csv')

,Dates,Prices,lower Prices,upper Prices
0,2020-10-31,10.100000,NaN,NaN
1,2020-11-01,10.095253,NaN,NaN
2,2020-11-02,10.091293,NaN,NaN
3,2020-11-03,10.088121,NaN,NaN
4,2020-11-04,10.085735,NaN,NaN
...,...,...,...,...
1791,2025-09-26,12.147848,11.578400,12.717295
1792,2025-09-27,12.168522,11.598418,12.738626
1793,2025-09-28,12.189884,11.619123,12.760645
1794,2025-09-29,12.211933,11.640512,12.783355


In [102]:

def calculate_contract_value(injection_dates, withdrawal_dates, volume):
    
    """Parameters

    Returns:
        injection_date: Date when commodity is sent for storage
        withdrawal_date: Date when commodity is withdrawn
        buy_price: Cost price per MMBtu
        sell_price: Selling price per MMBtu
        Volume: Volume of commodity traded
    """
    
    # Fixed values
    storage_cost_per_month = 100_000
    injection_withdrawal_cost = 10_000
    transport_cost = 50_000
    
    total_storage_duration = 0
    total_injection_withdrawal_cost = injection_withdrawal_cost * (len(injection_dates)+1) * (len(withdrawal_dates)+1)
    total_buy_cost = 0
    total_sell_revenue = 0
    
    for i in range(len(injection_dates)):
        in_date = injection_dates[i]
        out_date = withdrawal_dates[i]
        total_storage_duration += (out_date.year - in_date.year) * 12 + (out_date.month - in_date.month)
        
        buy_price = data[data['Dates']==str(in_date)]['Prices'].values[-1]
        total_buy_cost += buy_price * volume
        print(f"Price on {in_date}: {data[data['Dates']==str(in_date)]['Prices'].values[-1]}")
        
        sell_price = data[data['Dates']==str(out_date)]['Prices'].values[-1]
        total_sell_revenue += sell_price * volume
        print(f"Price on {out_date}: {data[data['Dates']==str(out_date)]['Prices'].values[-1]}")
                
    # Calculate costs and revenue
    total_storage_cost = storage_cost_per_month * total_storage_duration
    total_injection_withdrawal_cost = total_injection_withdrawal_cost * (volume / 1_000_000)  # For both injection and withdrawal
    total_transport_cost = transport_cost * 2  # For to and from transport
    
    # Calculate net profit
    net_profit = total_sell_revenue - total_buy_cost - total_storage_cost - total_injection_withdrawal_cost - total_transport_cost

    return np.round(net_profit,2)

In [103]:
print(calculate_contract_value(
    injection_dates=[date(2022, 1, 1), date(2022, 2, 1), date(2022, 2, 21), date(2022, 4, 1)], 
    withdrawal_dates=[date(2022, 1, 27), date(2022, 2, 15), date(2022, 3, 20), date(2022, 6, 1)], 
    volume=1_000_000
))

Price on 2022-01-01: 11.400740046452624
Price on 2022-01-27: 11.468350319248154
Price on 2022-02-01: 11.50906669738608
Price on 2022-02-15: 11.683825030463495
Price on 2022-02-21: 11.755261369958324
Price on 2022-03-20: 11.69888219463267
Price on 2022-04-01: 11.476909093432209
Price on 2022-06-01: 10.695717987428402
-1245201.68
